In [1]:
import KGInt as kg
import numpy as np
import h5py as h5
import matplotlib.pyplot as plt
from ipyparallel import Client

%matplotlib inline

In [2]:
f = h5.File("Levy_new.h5", "r")

limits = f["limits"]["0"].value
alphas = np.linspace(f["alphas"]["min_alpha"].value,f["alphas"]["max_alpha"].value, f["alphas"]["length"].value)
dr     = f["dr"]["0"].value

In [3]:
l=kg.Levyd(1., alphas[0], limits[0], limits[1], dr, f["Lr"]["0"].value, 1e-8)
c = kg.Coulomb2d(l)

In [4]:
c.set_R(5)

In [5]:
def integrate(ks, cs):
    c = cs[0]
    res = []
    for k in ks:
        res.append(c.integrate(k,1000000))
    return res

In [8]:
client = Client()
dv = client[:]

In [37]:
N       = 100
workers = len(client.ids)

if np.ceil(N/workers)!=np.floor(N/workers):
    print("number of nodes not compatible with the workers")

In [38]:
ks = [i/40 for i in range(1,N+1)]
cs = [c]*N

In [39]:
data = list(zip(*[iter(list(zip(ks,cs)))]*int(N/workers)))

In [ ]:
res = dv.map_async(integrate, data)